In [2]:
import os
import logging
import pandas as pd
import openpyxl
import json
from typing import Optional  # Add this import to handle Optional types


logging.basicConfig(level=logging.INFO)

def read_excel_file(file_path: str, sheet_name: Optional[str] = None) -> pd.DataFrame:
    """
    Read an Excel file or a specific sheet into a Pandas DataFrame.
    
    Args:
        file_path (str): The path to the Excel file.
        sheet_name (Optional[str]): The name of the sheet to read. If not provided, the first sheet will be used.
    
    Returns:
        pd.DataFrame: The DataFrame containing the Excel data.
    """
    try:
        return pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
    except Exception as e:
        logging.error(f"Error reading Excel file: {e}")
        raise

def convert_to_json(df: pd.DataFrame, output_file: str) -> None:
    """
    Convert a Pandas DataFrame to JSON and save it to a file.

    Args:
        df (pd.DataFrame): The DataFrame to convert.
        output_file (str): The path to the output JSON file.
    """
    try:
        json_data = df.to_json(orient='records')
        with open(output_file, 'w') as f:
            json.dump(json_data, f, indent=4)
        logging.info(f"JSON data exported to {output_file}")
    except Exception as e:
        logging.error(f"Error converting to JSON: {e}")

def print_workbook_properties(workbook: openpyxl.Workbook) -> None:
    """
    Print the properties of an openpyxl Workbook.
    """
    properties = workbook.properties
    logging.info(f"Title: {properties.title}")
    logging.info(f"Creator: {properties.creator}")
    logging.info(f"Created: {properties.created}")
    logging.info(f"Modified: {properties.modified}")
    logging.info(f"Last Modified By: {properties.lastModifiedBy}")

def print_sheet_formulas(sheet: openpyxl.worksheet.worksheet.Worksheet) -> None:
    """
    Print the formulas in a given sheet.
    """
    logging.info(f"Formulas in sheet '{sheet.title}':")
    for row in sheet.iter_rows():
        for cell in row:
            if isinstance(cell.value, str) and cell.value.startswith('='):
                logging.info(f"Cell {cell.coordinate} contains the formula: {cell.value}")

def convert_to_numeric(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert object columns to numeric where possible.
    
    Args:
        df (pd.DataFrame): The input DataFrame.
    
    Returns:
        pd.DataFrame: The DataFrame with object columns converted to numeric.
    """
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def main(excel_file_path: Optional[str] = None) -> None:
    """
    Main function to execute the Excel file operations.
    
    Args:
        excel_file_path (Optional[str]): The path to the Excel file. If not provided, a default path will be used.
    """
    if not excel_file_path:
        excel_file_path = 'Pista Evaluación final.xlsx'

    # Load the workbook using openpyxl
    workbook = openpyxl.load_workbook(excel_file_path, data_only=False)

    # Print workbook properties
    print_workbook_properties(workbook)

    # Inspecting WACC and Resultados sheets
    wacc_sheet = workbook['WACC']
    resultado_sheet = workbook['7.Resultado']

    logging.info("\nFormulas in 'WACC' sheet:")
    print_sheet_formulas(wacc_sheet)

    logging.info("\nFormulas in '7.Resultado' sheet:")
    print_sheet_formulas(resultado_sheet)

    # Read and clean the '6. Datos' sheet
    df = read_excel_file(excel_file_path, sheet_name='6. Datos')
    cleaned_df = convert_to_numeric(df)

    # Log data types after conversion
    logging.info("\nData Types After Conversion:")
    logging.info(cleaned_df.dtypes)

    # Log missing values count
    logging.info("\nMissing Values Count:")
    logging.info(cleaned_df.isnull().sum())

    # Convert DataFrame to JSON
    convert_to_json(cleaned_df, 'output.json')

if __name__ == "__main__":
    main()


INFO:root:Title: None
INFO:root:Creator: Master
INFO:root:Created: 2015-07-05 20:50:13
INFO:root:Modified: 2024-09-10 10:41:33
INFO:root:Last Modified By: Maria Isabel Jara López
INFO:root:
Formulas in 'WACC' sheet:
INFO:root:Formulas in sheet 'WACC':
INFO:root:Cell F22 contains the formula: =G8*(G16/(G16+G18))+G6*(G18/(G16+G18))
INFO:root:Cell F23 contains the formula: =G8*(G17/(G17+G18))+G6*(G18/(G17+G18))
INFO:root:
Formulas in '7.Resultado' sheet:
INFO:root:Formulas in sheet '7.Resultado':
INFO:root:Cell D5 contains the formula: =+C5+(D18*7%)
INFO:root:Cell D6 contains the formula: =+D83
INFO:root:Cell D7 contains the formula: =+D82
INFO:root:Cell C9 contains the formula: =SUM(C5:C8)
INFO:root:Cell D9 contains the formula: =SUM(D5:D8)
INFO:root:Cell C15 contains the formula: =SUM(C11:C14)
INFO:root:Cell D15 contains the formula: =SUM(D11:D14)
INFO:root:Cell D18 contains the formula: =+(C18*1.06)
INFO:root:Cell C19 contains the formula: =C18-C20
INFO:root:Cell D19 contains the formu